<a href="https://colab.research.google.com/github/thdnjs0730/2024-1-my-BDA/blob/main/project/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EA%B3%BC%EC%A0%9C_03%EC%A1%B0_20231596_%EC%B5%9C%EC%86%8C%EC%9B%90_20231596_%EC%B5%9C%EC%86%8C%EC%9B%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 청소년 정신건강 지원 프로그램 제안

## 1. 데이터 로드 및 전처리

In [1]:
import pandas as pd

# 데이터 로드
depression_treatment_df = pd.read_csv('우울증 치료 경험.csv')
youth_depression_rate_df = pd.read_csv('청소년 우울감 경험률.csv')
population_data_df = pd.read_csv('202403_202403_주민등록 인구 기타현황(아동청소년청년 인구현황)_ChYouth.csv')

FileNotFoundError: [Errno 2] No such file or directory: '우울증 치료 경험.csv'

In [ ]:
depression_treatment_df.head()

In [ ]:
youth_depression_rate_df.head()

In [ ]:
population_data_df.head()

### '우울증 치료 경험' 데이터 전처리

In [ ]:
# 모든 값이 NaN인 행을 제거, 인덱스 재설정
depression_treatment_df = depression_treatment_df.dropna(how='all').reset_index(drop=True)

# 헤더 행을 식별하고 해당 행을 열 이름으로 설정
header_row_index = depression_treatment_df.index[depression_treatment_df.iloc[:, 0] == '구분'][0]
depression_treatment_df.columns = depression_treatment_df.iloc[header_row_index]

# 헤더 행 및 그 이전의 행들을 제거하고 인덱스 재설정
depression_treatment_df = depression_treatment_df.drop(range(header_row_index + 1)).reset_index(drop=True)

# 불필요한 행 제거 (우울증, 조울증, 조현병 데이터만 남김)
depression_treatment_df_cleaned = depression_treatment_df[depression_treatment_df['구분'].isin(['우울증', '조울증', '조현병'])]

# 열 이름 변경
depression_treatment_df_cleaned.columns = ['구분', '2018 환자수', '2018 진료건수', '2019 환자수', '2019 진료건수',
                                           '2020 환자수', '2020 진료건수', '2021 환자수', '2021 진료건수',
                                           '2022 환자수', '2022 진료건수', '2023 환자수', '2023 진료건수']

# 숫자 값 정수형 변환
for col in depression_treatment_df_cleaned.columns[1:]:
    depression_treatment_df_cleaned[col] = depression_treatment_df_cleaned[col].astype(str).str.replace(',', '').astype(int)

# 인덱스 재설정 및 데이터 프레임 출력
depression_treatment_df_cleaned.reset_index(drop=True, inplace=True)
depression_treatment_df_cleaned.head()

### '청소년 우울감 경험률' 데이터 전처리

In [ ]:
# 인덱스 재설정
youth_depression_rate_df.columns = ['연도', '중학생 우울감 경험률', '고등학생 우울감 경험률']

# 정수형을 실수형으로 변환
youth_depression_rate_df['연도'] = youth_depression_rate_df['연도'].astype(int)
youth_depression_rate_df['중학생 우울감 경험률'] = youth_depression_rate_df['중학생 우울감 경험률'].astype(float)
youth_depression_rate_df['고등학생 우울감 경험률'] = youth_depression_rate_df['고등학생 우울감 경험률'].astype(float)

# 데이터 프레임 출력
youth_depression_rate_df.head()

### '주민등록 인구 현황 데이터 전처리'

In [ ]:
# 4번째 열부터 모든 열을 제거
population_data_df_cleaned = population_data_df.iloc[:, :3]

# 열 이름 변경
population_data_df_cleaned.columns = ['행정기관코드', '행정기관', '청소년 전체']

# 모든 값이 NaN인 행을 제거, 인덱스 재설정
population_data_df_cleaned = population_data_df_cleaned.dropna(how='all').reset_index(drop=True)

# 숫자 값을 정수형으로 변환
population_data_df_cleaned['청소년 전체'] = population_data_df_cleaned['청소년 전체'].astype(str).str.replace(',', '').astype(int)

# 데이터 프레임 출력
population_data_df_cleaned.head()

## 2. 연도별 비교

### 연도별 데이터 통합

In [ ]:
# Select relevant columns and rows
population_data_df = population_data_df[['행정기관', '청소년 전체']]

# Drop rows with NaN values and reset the index
population_data_df = population_data_df.dropna().reset_index(drop=True)

# Convert population values to integers after removing commas
population_data_df['청소년 전체'] = population_data_df['청소년 전체'].astype(str).str.replace(',', '').astype(int)

# Aggregate population data by year if needed (assuming the dataset contains such information)
# For now, let's assume the data represents a specific recent year, and summarize the total population
total_population = population_data_df['청소년 전체'].sum()

# Display the cleaned dataframe and total population
population_data_df.head()

,행정기관,청소년 전체
0,서울특별시,1353501
1,서울특별시 서울특별시 종로구,19718
2,서울특별시 서울특별시 중구,13415
3,서울특별시 서울특별시 용산구,26090
4,서울특별시 서울특별시 성동구,36886


### 연도별 청소년 우울감 경험률, 우울증 환자 비율 비교

In [ ]:
# 각 질환, 연도에 대한 관련 데이터 추출
depression_data = depression_treatment_df_cleaned[depression_treatment_df_cleaned['구분'] == '우울증'].iloc[0, 1:].values
bipolar_data = depression_treatment_df_cleaned[depression_treatment_df_cleaned['구분'] == '조울증'].iloc[0, 1:].values
schizophrenia_data = depression_treatment_df_cleaned[depression_treatment_df_cleaned['구분'] == '조현병'].iloc[0, 1:].values

In [ ]:
# 데이터를 환자 수와 진료 건수로 분할
depression_patients = depression_data[::2]
depression_consultations = depression_data[1::2]
bipolar_patients = bipolar_data[::2]
bipolar_consultations = bipolar_data[1::2]
schizophrenia_patients = schizophrenia_data[::2]
schizophrenia_consultations = schizophrenia_data[1::2]

In [ ]:
# 새로운 데이터 프레임 생성
merged_df = pd.DataFrame({
    '연도': [2018, 2019, 2020, 2021, 2022, 2023],
    '우울증 환자수': depression_patients,
    '우울증 진료건수': depression_consultations,
    '조울증 환자수': bipolar_patients,
    '조울증 진료건수': bipolar_consultations,
    '조현병 환자수': schizophrenia_patients,
    '조현병 진료건수': schizophrenia_consultations
})

In [ ]:
# 청소년 우울감 경험률 데이터와 병합
merged_df = pd.merge(merged_df, youth_depression_rate_df, left_on='연도', right_on='연도', how='inner')

# 청소년 전체 인구 값을 데이터 프레임에 추가
merged_df['청소년 전체 인구'] = total_population

# 인구 10만명당 우울증 환자 비율 계산
merged_df['우울증 환자 비율 (인구 10만명당)'] = (merged_df['우울증 환자수'] / merged_df['청소년 전체 인구']) * 100000

# 데이터 프레임 출력
merged_df.head()

,연도,우울증 환자수,우울증 진료건수,조울증 환자수,조울증 진료건수,조현병 환자수,조현병 진료건수,중학생 우울감 경험률,고등학생 우울감 경험률,청소년 전체 인구,우울증 환자 비율 (인구 10만명당)
0,2018,752976,6746590,95911,1052788,121058,1600480,25.2,28.7,2707002,27815.864192
1,2019,799011,7379918,106170,1188329,121158,1620629,26.9,29.4,2707002,29516.453996
2,2020,832378,7890603,111854,1269495,119719,1560222,22.9,27.4,2707002,30749.072221
3,2021,915298,8864640,120192,1375367,120856,1555797,25.9,27.7,2707002,33812.239518
4,2022,1000744,9500696,130589,1469299,120722,1542962,28.2,29.3,2707002,36968.720378
